In [2]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling, timeframes
import numpy as np
from datetime import datetime, timedelta



In [3]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./half_month_collections.pkl")

In [5]:
print(f"Amount of assets : {len(L)}")
print(f"Amount of meters : {len(M)}")
print(f"Amount of hours : {len(H)} and amount of days : {len(H)/24}")

Amount of assets : 2189
Amount of meters : 62
Amount of hours : 408 and amount of days : 17.0


In [ ]:
markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())

In [ ]:
def find_previous_weekday(d, weekday):
    days_behind = d.weekday() - weekday
    if days_behind < 0: # Target day is in the previous week
        days_behind += 7
    return d - timedelta(days=days_behind)

# Function to calculate the deadlines for a given input timestamp
def calculate_deadlines_for_input(input_timestamp):
    deadlines = {}

    # RKOM uke: Friday the week before if on a weekday, otherwise Thursday the same week
    if input_timestamp.weekday() < 5:  # Weekday
        deadlines['RKOM uke Weekday'] = find_previous_weekday(input_timestamp - timedelta(days=7), 4)  # Previous Friday
    else:  # Weekend
        deadlines['RKOM uke Weekend'] = find_previous_weekday(input_timestamp, 3)  # Current Thursday

    # FCR D-2: Two days before the input time at 17:30
    deadlines['FCR D-2'] = datetime.combine(input_timestamp.date() - timedelta(days=2), datetime.strptime('17:30', '%H:%M').time())

    # aFRR CM: One day before the input time at 07:30
    deadlines['aFRR'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('07:30', '%H:%M').time())

    # FCR D-1: One day before the input time at 18:00
    deadlines['FCR D-1'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('18:00', '%H:%M').time())

    # RK: 45 minutes before the input time
    deadlines['RK'] = input_timestamp - timedelta(minutes=45)

    return deadlines

In [ ]:
type(H[3])

In [ ]:
def get_possible_dates(date : pd.Timestamp):
    
    if date.hour == 17: # FCR D-2
        return (pd.date_range(date + timedelta(days=1) + timedelta(hours=7), date + timedelta(days = 2) + timedelta(hours = 6), freq='H', tz = "Europe/Oslo"), "D_2")
    elif date.hour == 7: # aFRR
        return (pd.date_range(date + timedelta(hours = 17), date + timedelta(days = 1) + timedelta(hours = 16), freq='H', tz = "Europe/Oslo"), "aFRR")
    elif date.hour == 18: # FCR D-1
        return (pd.date_range(date + timedelta(hours=6), date + timedelta(days = 1) + timedelta(hours = 5), freq='H', tz = "Europe/Oslo"), "D_1")
    else:
        return ([], "No bids")
    
    """elif date.day_of_week == 4: # friday
        if date.hour == 12: #RKOM uke
            # get hours from monday next week until friday next week
            week_dates = pd.date_range(date + timedelta(days=2) + timedelta(hours = 12), periods = 5, freq = 'D', tz = "Europe/Oslo") #RKOM uke weekdays night
            day_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(5, 24)]
            night_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(0, 5)]
            return ((day_hours, night_hours), ("RKOM_uke weekday day", "RKOM_uke weekday night"))
        else:
            return ([], "No bids")
    elif date.day_of_week == 3: #thursday
        if date.hour == 12:
            week_dates = pd.date_range(date + timedelta(days=1), periods = 2, freq='D', tz = "Europe/Oslo") #RKOM uke weekend night
            day_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(5, 24)]
            night_hours = [pd.Timestamp(year=date.year, month=date.month, day=date.day, hour=hour) for date in week_dates for hour in range(0, 5)]
            return ((day_hours, night_hours), ("RKOM_uke weekend day", "RKOM_uke weekend night"))
        else:
            return ([], "No bids")"""

In [ ]:
H[0].day

#### The bids doesnt have to be for all hours within a day - can choose to bid for only chosen hours. There is no constraints due to anything but the market constraints. Can be for any number of hours within a day and dont have to be connected hours.

#### The strategy should probably be to bid for the hours where the expected value is highest. If a bid is accepted for an hour in a market it will not be able to bid for the same hour in another market.

In [ ]:
def get_possible_bids(possible_hours, available_assets, possible_markets):
    """ Function to find the possible bids for the given input hours, assets and markets. In this version the possible bids are constrained to be for every one of the 24 hours.

    Args:
        possible_hours (DateTimeIndex): list of the possible hours where bids can be placed
        available_assets (list(new_meters.PowerMeter)): list of assets that are not bid in to other markets in the possible hours
        possible_markets (list(final_markets.ReserveMarket)): _description_

    Returns:
        dict: dictionary with the possible bids for each market where the keys are tuples of the market name and the date and the values are tuples of the possible revenues and the assets that can bid in the market
    """
    possible_bids = {}
    date = possible_hours[0].day
    for market in possible_markets:
        if market.direction == "up":
            possible_assets = [asset for asset in available_assets if asset.area == market.area and asset.direction != "down"]
            possible_volumes = np.array([np.sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets]) for hour in possible_hours]) 
            if (possible_volumes > market.min_volume).all(): # this
                possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
                possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
                possible_bids[(market.name, date)] = (possible_revenues, possible_assets)
        elif market.direction == "down":
            possible_assets = [asset for asset in available_assets if asset.area == market.area and asset.direction != "up"]
            possible_volumes = np.array([np.sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets]) for hour in possible_hours]) 
            if (possible_volumes > market.min_volume).all():
                possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
                possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
                possible_bids[(market.name, date)] = (possible_revenues, possible_assets)
        else:
            possible_assets = [asset for asset in available_assets if asset.area == market.area]
            indices = [H.get_loc(ts) for ts in possible_hours]
            directions = np.array(dominant_directions[indices[0]:indices[-1]+1])
            mask_up = directions == "up"
            mask_down = directions == "down"
            possible_up_volumes = np.array([np.sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets]) for hour in possible_hours]) 
            possible_down_volumes = np.array([np.sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets]) for hour in possible_hours]) 
            #print(f"possible_up_volumes: {possible_up_volumes}")
            #print(f"possible_down_volumes: {possible_down_volumes}")
            possible_volumes = np.where(mask_up, possible_up_volumes, possible_down_volumes)
            if (possible_volumes > market.min_volume).all():
                possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
                possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
                possible_bids[(market.name, date)] = (possible_revenues, possible_assets)
    return possible_bids
    


In [ ]:
def place_bids(market, available_assets, possible_hours ):
    if market.direction == "up":
        # check the compatibility for the assets
        possible_assets = [asset for asset in available_assets if asset.area == market.area and asset.direction != "down"]
        #get the volumes in the correct direction
        possible_volumes = np.array([np.sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets]) for hour in possible_hours]) 
        # get the prices for the given market within the given hours
        
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        # Calculate the possible revenues
        possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
        # Place the bids in the dictionary
        return (possible_revenues, possible_assets)
    elif market.direction == "down":
        possible_assets = [asset for asset in available_assets if asset.area == market.area and asset.direction != "up"]
        possible_volumes = np.array([np.sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets]) for hour in possible_hours]) 
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
        return (possible_revenues, possible_assets)
    else:
        possible_assets = [asset for asset in available_assets if asset.area == market.area]
        indices = [H.get_loc(ts) for ts in possible_hours]
        directions = np.array(dominant_directions[indices[0]:indices[-1]+1])
        mask_up = directions == "up"
        mask_down = directions == "down"
        possible_up_volumes = np.array([np.sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets]) for hour in possible_hours]) 
        possible_down_volumes = np.array([np.sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets]) for hour in possible_hours]) 
        #print(f"possible_up_volumes: {possible_up_volumes}")
        #print(f"possible_down_volumes: {possible_down_volumes}")
        possible_volumes = np.where(mask_up, possible_up_volumes, possible_down_volumes)
        possible_prices = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]] for hour in possible_hours]) # set of prices for markets
        possible_revenues = np.sum(possible_prices * possible_volumes, axis = 1)
        return (possible_revenues, possible_assets)

In [ ]:
def get_possible_bids_unconstrained(possible_hours : [pd.Timestamp], available_assets : [new_meters.PowerMeter], possible_markets : [final_markets.ReserveMarket]):
    """ Function to find the possible bids for the given input hours, assets and markets. where the possible bids are not constrained to be for every one of the 24 hours.

    Args:
        possible_hours (DateTimeIndex): list of the possible hours where bids can be placed
        available_assets ([new_meters.PowerMeter]): list of assets that are not bid in to other markets in the possible hours
        possible_markets ([final_markets.ReserveMarket]): _description_

    Returns:
        dict: dictionary with the possible bids for each market where the keys are tuples of the market name and the date and the values are tuples of the possible revenues and the assets that can bid in the market
    """
    possible_bids = {}
    date = possible_hours[0].day
    for market in possible_markets:
        possible_bids[(market.name, date)] = place_bids(possible_hours, available_assets, market)
    return possible_bids

In [118]:
H[3]

Timestamp('2023-06-14 03:00:00+0200', tz='Europe/Oslo')

In [119]:
bid_hours, market_name = get_possible_dates(H[7])

In [121]:
str(bid_hours)

"DatetimeIndex(['2023-06-15 00:00:00+02:00', '2023-06-15 01:00:00+02:00',\n               '2023-06-15 02:00:00+02:00', '2023-06-15 03:00:00+02:00',\n               '2023-06-15 04:00:00+02:00', '2023-06-15 05:00:00+02:00',\n               '2023-06-15 06:00:00+02:00', '2023-06-15 07:00:00+02:00',\n               '2023-06-15 08:00:00+02:00', '2023-06-15 09:00:00+02:00',\n               '2023-06-15 10:00:00+02:00', '2023-06-15 11:00:00+02:00',\n               '2023-06-15 12:00:00+02:00', '2023-06-15 13:00:00+02:00',\n               '2023-06-15 14:00:00+02:00', '2023-06-15 15:00:00+02:00',\n               '2023-06-15 16:00:00+02:00', '2023-06-15 17:00:00+02:00',\n               '2023-06-15 18:00:00+02:00', '2023-06-15 19:00:00+02:00',\n               '2023-06-15 20:00:00+02:00', '2023-06-15 21:00:00+02:00',\n               '2023-06-15 22:00:00+02:00', '2023-06-15 23:00:00+02:00'],\n              dtype='datetime64[ns, Europe/Oslo]', freq='H')"

In [131]:
def make_bids_for_one_week(L,M,H):
    """ Function to make bids for one week. 
    The function iterates through the hours in the week and finds the possible bids for each hour. 
    The function then chooses the bid with the highest revenue and adds it to the list of bids. 
    The function also keeps track of the assets that are active in their respective markets for the respective time period.
   

    Args:
        L (list(new_meters.PowerMeter)): _description_
        M (list(final_markets.ReserveMarket)): _description_
        H (_type_): _description_

    Returns:
        _type_: _description_
    """
    #unactive_assets = L.copy()
    active_assets = {}
    revenue = 0
    active_bids = {}
    bid_list = []
    for hour in list(H)[:168]: #19-26.06.2023
        bid_hours, market_name = get_possible_dates(hour)
        if len(bid_hours) == 0:
            continue
        else:
            possible_markets = [m for m in M if market_name in m.name]
            #print(f"Possible markets: {[m.name for m in possible_markets]}")
            if str(bid_hours) in active_assets.keys():
                possible_assets = [asset for asset in L if asset not in active_assets[str(bid_hours)]]
            else:
                possible_assets = [asset for asset in L]
                
            possible_bids = get_possible_bids(bid_hours, possible_assets, possible_markets)
            # possible_bids is a dictionary with keys (market_name, date) and values (possible_revenues, possible_assets)
            if len(possible_bids) > 1:
                revenues = [sum(possible_bids[key][0]) for key in possible_bids.keys()]
                print(f"revenues: {revenues}")
                best_bid_revenue = max(revenues)
                #bid_list.append(best_bid)
                revenue += best_bid_revenue
                best_bid_key = list(possible_bids.keys())[revenues.index(best_bid_revenue)]
                best_bid_assets = possible_bids[best_bid_key][1]
                active_bids[best_bid_key] = best_bid_assets
                active_assets[str(bid_hours)] = best_bid_assets
                #unactive_assets = [asset for asset in unactive_assets if asset not in best_bid_assets]
                #print(f"active assets : { active_assets}")
            elif len(possible_bids) == 1:
                #bid_list.append(possible_bids[list(possible_bids.keys())[0]][0])
                print("revenue to take sum on : ", possible_bids[list(possible_bids.keys())[0]][0])
                revenue += sum(possible_bids[list(possible_bids.keys())[0]][0])
                best_bid_key = list(possible_bids.keys())[0]
                best_bid_assets = possible_bids[best_bid_key][1]
                active_bids[best_bid_key] = best_bid_assets
                active_assets[str(bid_hours)] = best_bid_assets
                #unactive_assets = [asset for asset in unactive_assets if asset not in best_bid_assets]
            else:
                continue
    return bid_list, revenue, active_bids, active_assets

In [132]:
bl, rev, ab = make_bids_for_one_week(L,M,H)

revenues: [34792.61560481999, 51400.56588749998]
revenues: [0.0, 0.0, 51374.372821199984]
revenue to take sum on :  [ 943.41044       0.            0.            0.            0.
 1275.49091488 1347.7292     1364.44104208 1364.44104208  943.41044
  943.41044     943.41044    1288.4291152  1288.4291152  1288.4291152
 1132.092528    943.41044     943.41044     943.41044     943.41044
  943.41044     889.501272    808.63752     835.592104  ]
revenues: [0.0, 51434.88181624999]
revenues: [24824.536052940006, 32288.613687449997]
revenues: [0.0, 74174.27187963]
revenues: [0.0, 0.0, 31851.939204]
revenue to take sum on :  [ 980.89068      0.           0.           0.           0.
 1128.024282  1078.979748  1078.979748  1078.979748   980.89068
  980.89068   1005.412947  1078.979748  1078.979748  1078.979748
  985.7951334  980.89068    882.801612   882.801612   882.801612
  882.801612   882.801612   882.801612   980.89068  ]
revenues: [0.0, 24080.0633682]
revenues: [20934.523230040006, 34444.911

In [137]:
print(7)